## **ANALYSIS_masters_make_Fst_Fst_matrix**

### GOALS of this script:
* calculate Hudson FstxFst matrix for 35 landraces 
* make heatmap with/without dendrogram


> #### **currently the pairwise Fst only runs on the 25 Mayer_2017 landraces**

> #### **make sure that SNPs on Chr0 is not included - maybe in the future currently not priority**

### **Import packages**

In [ ]:
import numpy as np
import seaborn as sns
sns.set_style('white')
sns.set_style('ticks')
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import allel
import random
import h5py
import bcolz
import csv
from itertools import chain
import itertools
import os
from collections import Counter
from itertools import combinations
import seaborn as sns; sns.set_theme(color_codes=True)
from matplotlib import pyplot

### **Load the data**

In [ ]:
callset = allel.read_vcf('/Users/kschul38/Documents/tsinfer-project/data/2_processed/europe_maize_dataset_600k/merge_merge_all_chrom_all_edits_with_allel_column_vcf.vcf')

In [ ]:
#callset

In [ ]:
sorted(callset.keys())

In [ ]:
#get the samples in the .vcf
samples= callset["samples"]
#samples

In [ ]:
#total number of the samples inn the vcf
len(samples)

In [ ]:
#put all the samples in a Dataframe
sample_subset = pd.DataFrame(data=samples, columns = ['sample_names'])
#sample_subset

### **Get the names of the landraces from the .vcf file**

#### **Mayer_2017_landraces** - 952 individuals based on **35** landraces

In [ ]:
#all mayer_2017 sample names
mayer_2017_landraces_samples=sample_subset[sample_subset.sample_names.str.contains("_2017_landraces")]

In [ ]:
#number of all Mayer_2017_landraces
print(len(mayer_2017_landraces_samples))
#mayer_2017_landraces_samples

In [ ]:
mayer_2017_35_landraces=mayer_2017_landraces_samples[mayer_2017_landraces_samples.sample_names.str.contains("\\.01")]
print(len(mayer_2017_35_landraces))
#print(mayer_2017_35_landraces)

### **Metadata - Add the populations to the individuals**

#### **Mayer_2017_landraces** - 952 individuals based on **35** landraces

In [ ]:
for index, row in mayer_2017_landraces_samples.iterrows():
    #print(row['sample_names'])
    mayer_string=row['sample_names']#take the sample names row
    #print(mayer_string) 
    mayer_substring=mayer_string[0:2] #take the first two letters of the sample names
    #print(mayer_substring)
    mayer_2017_landraces_samples.at[index,'Population'] = mayer_substring + "_mayer_2017" #add the dataset info and make the Population column                                                                                                                                                     

In [ ]:
mayer_2017_landraces_samples

In [ ]:
#get all the unique population that you have assigned and check the number 
populations_mayer = mayer_2017_landraces_samples.Population.unique()
print(len(populations_mayer))
print(populations_mayer)

### **Get all combinations of the different populations for the pairwise-Fst**

#### **Get the names for the combinations**

In [ ]:
two_row_dataframe = {}
splits = list(mayer_2017_landraces_samples.groupby("Population"))
for i in range(35):
    two_row_dataframe[i] = [] 
    # view splitted dataframe
    #print(splits[i][1])
    two_row_dataframe[i]=splits[i][1]

In [ ]:
first_row_dataframe={} 
second_row_dataframe={}
for i in range(35):
    first_row_dataframe[i]=[]
    second_row_dataframe[i]=[]
    # view splitted dataframe
    #print(two_row_dataframe[i].sample_names)
    #print(two_row_dataframe[i].Population)
    first_row_dataframe[i]=two_row_dataframe[i].sample_names
    second_row_dataframe[i]=two_row_dataframe[i].Population

In [ ]:
#first_row_dataframe

In [ ]:
#second_row_dataframe

#### **Make the combinations by numbers**

In [ ]:
#make a list of the length 35 from 0 to 34
L=list(range(35))
print(len(L))
print(L)

In [ ]:
#make a list of all combinations - **including the reciprocal (O,1) and (1,0)** because they are needed for the heatmap
combo=list(itertools.permutations(L,2))
print(len(combo))
#print(combo)

In [ ]:
#append the self ones to the combo list 
combo.append((0,0))
combo.append((1,1))
combo.append((2,2))
combo.append((3,3))
combo.append((4,4))
combo.append((5,5))
combo.append((6,6))
combo.append((7,7))
combo.append((8,8))
combo.append((9,9))
combo.append((10,10))
combo.append((11,11))
combo.append((12,12))
combo.append((13,13))
combo.append((14,14))
combo.append((15,15))
combo.append((16,16))
combo.append((17,17))
combo.append((18,18))
combo.append((19,19))
combo.append((20,20))
combo.append((21,21))
combo.append((22,22))
combo.append((23,23))
combo.append((24,24))
combo.append((25,25))
combo.append((26,26))
combo.append((27,27))
combo.append((28,28))
combo.append((29,29))
combo.append((30,30))
combo.append((31,31))
combo.append((32,32))
combo.append((33,33))
combo.append((34,34))

In [ ]:
#sort the combo list
combo.sort()

In [ ]:
#check the final combo list
print(len(combo))
#print(combo)
1225-1190

### **Calculate the pairwise-Fst**

In [ ]:
combis=combo
combis.sort()
print(combis)
print(len(combis))

In [ ]:
combi_list= {}
for i in range(1225):
    combis_element=combis[i]
    combis_fist=combis_element[0]
    combis_second=combis_element[1]
    #print(combis_element)
    #print(combis_fist)
    #print(combis_second)
    list_first=(list(first_row_dataframe[combis_fist]))
    list_second=(list(first_row_dataframe[combis_second]))
    #print(list_first)
    #print(list_second)
    combi_list[i]=[]
    combi_list[i]=list_first+list_second
    #print(combi_list)
    #print(len(combi_list))

In [ ]:
#combi_list

In [ ]:
#set up the lists for the Fst values and the corresponding Fst standard deviations
fst_list=[]
fst_se_list=[]

In [ ]:
#set up a list of all Mayer_2017 landraces names for the print statement of the calculations
unique_populations_mayer=pd.unique(populations_mayer)
print(len(unique_populations_mayer))
print(unique_populations_mayer)

In [ ]:
#calculate the pairwise Fst values and corresponding deviations 
for k in range(1225):
    combis_element=combis[k]
    combis_fist=combis_element[0]
    first_landrace_name=unique_populations_mayer[combis_fist]
    combis_second=combis_element[1]
    second_landrace_name=unique_populations_mayer[combis_second]
    combi_list_element=combi_list[k]
    #print(combis_element)
    #print(combis_fist)
    #print(combis_second)
    #print(first_landrace_name)
    #print(second_landrace_name)
    #print(combi_list_element)
    
    #load the data for the two landraces seperatly 
    loc_samples_pop1 = allel.read_vcf('/Users/kschul38/Documents/tsinfer-project/data/2_processed/europe_maize_dataset_600k/merge_merge_all_chrom_all_edits_with_allel_column_vcf.vcf', samples= first_row_dataframe[combis_fist])
    loc_samples_pop2 = allel.read_vcf('/Users/kschul38/Documents/tsinfer-project/data/2_processed/europe_maize_dataset_600k/merge_merge_all_chrom_all_edits_with_allel_column_vcf.vcf', samples= first_row_dataframe[combis_second])
    
    #load the data for the combination of the two samples 
    loc_samples_combi_pops = allel.read_vcf('/Users/kschul38/Documents/tsinfer-project/data/2_processed/europe_maize_dataset_600k/merge_merge_all_chrom_all_edits_with_allel_column_vcf.vcf', samples= combi_list_element)  
    
    #get the genotype data for the two landraces seperatly
    geno_samples_pop1=allel.GenotypeArray(loc_samples_pop1['calldata/GT'])
    geno_samples_pop2=allel.GenotypeArray(loc_samples_pop2['calldata/GT'])
    
    #get the genotype data for the combination
    geno_samples_combi_pop=allel.allel.GenotypeArray(loc_samples_combi_pops['calldata/GT'])
    
    #get the allel count array for the two landraces seperatly
    geno_samples_pop1_allel_count=geno_samples_pop1.count_alleles()
    geno_samples_pop2_allel_count=geno_samples_pop2.count_alleles()
    
    #get the allel count array for the combination of the two landraces
    geno_samples_combi_pop_allel_count=geno_samples_combi_pop.count_alleles()
    
    #boolean to check if the selected individuals segregeate at the SNPs
    is_seg = geno_samples_combi_pop_allel_count.is_segregating()[:]
    
    #apply the boolean to the allel count arrays 
    geno_samples_pop1_allel_count_seg=geno_samples_pop1_allel_count.compress(is_seg, axis=0)
    geno_samples_pop2_allel_count_seg=geno_samples_pop2_allel_count.compress(is_seg, axis=0)
    
    
    #calculate the Fst
    fst, fst_se, _, _ = allel.blockwise_hudson_fst(geno_samples_pop1_allel_count_seg,geno_samples_pop2_allel_count_seg, blen=100000)
    
    print(fst)
    print(fst_se)
    #append fst and fst_se to list
    fst_list.append(fst)
    fst_se_list.append(fst_se)
    
    #print("Fst for %s combination - landraces %s/%s was calculated" % (str(combis_element)))
    print("Fst for %(a)s combination - landraces %(b)s/%(c)s was calculated" % {'a': str(combis_element), 'b': first_landrace_name, 'c':second_landrace_name})
    

In [ ]:
#check the length of the Fst value list - should be 1225 (35*35)
len(fst_list)

In [ ]:
xxxxx

#### **Write fst values to file**

In [ ]:
#print(fst_list)

In [ ]:
textfile = open("fst_hudson_list_for_masters.txt", "w")
for element in fst_list:
    textfile.write(str(element)+ "\n")
textfile.close()

#### **Load random values for FstxFst matrix to check quit edits**

In [ ]:
randomlist =  np.random.uniform(0.0,1.0, 1225)
print(len(randomlist))
fst_list=randomlist

### **Make FstxFst matrix heatmap plot**

#### **Shape fst values to fit the heatmap**

In [ ]:
#round to three decimals
fst_rounded=np.around(fst_list, decimals=3)

In [ ]:
#replace the negative values from the self calculation with 0es
fst_rounded[fst_rounded < 0] = 0

In [ ]:
#split array into arrays with length of populations 35 for each column of the heatmap
split_fst_array=np.split(fst_rounded,35)

In [ ]:
#stack the 35 length arrays for the heatmap
split_fst_stacks=np.stack(split_fst_array)

In [ ]:
split_fst_stacks

#### **Set up the names for the y and x-axis of the heatmap**

In [ ]:
#set up the lists 
labels=[]
y_labels=[]
x_labels=[]

In [ ]:
#shorten the pop names to the abbreviations 
for first_two in populations_mayer:
    #print(first_two[:2])
    labels.append(first_two[:2])

In [ ]:
y_labels= labels
x_labels= labels

In [ ]:
#check what the axis labels contain
#print(y_labels)
#print(x_labels)

In [ ]:
#turn around one of the axis labels 
y_labels=x_labels[::-1]

In [ ]:
#check the orientations 
print(y_labels)
print(x_labels)

In [ ]:
rev_split_fst_stacks=[]

In [ ]:
split_fst_stacks[34]

In [ ]:
for i in range(35):
    num=34-i
    print(num)
    rev_split_fst_stacks.append(split_fst_stacks[num])
    

In [ ]:
#rev_split_fst_stacks

#### **Make the heatmap plot**

In [ ]:
fst_heat_values = split_fst_stacks

In [ ]:
cbarlabel="Fst value"

In [ ]:
fig, ax = plt.subplots()
im = ax.imshow(fst_heat_values)


# Create colorbar
cbar = ax.figure.colorbar(im, ax=ax)
cbar.ax.set_ylabel(cbarlabel,rotation=-90, va="bottom",fontsize=25)


# We want to show all ticks...
ax.set_xticks(np.arange(len(x_labels)))
ax.set_yticks(np.arange(len(y_labels)))
# ... and label them with the respective list entries
ax.set_xticklabels(x_labels, size=20)
ax.set_yticklabels(y_labels, size=20)
ax.grid(False) #remove the gridlines

# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

# Loop over data dimensions and create text annotations.
for i in range(len(y_labels)):
    for j in range(len(x_labels)):
        text = ax.text(j, i, fst_heat_values[i, j],
                       ha="center", va="center", color="white",size=12,fontweight="bold")

#ax.set_title("Pairwise Fst",fontsize=40)
#fig.tight_layout()
fig.set_size_inches(30, 30)
#fig.savefig('full_heatmap_without_dendrogram.png')
plt.show()

In [ ]:
df= pd.DataFrame(data=split_fst_stacks)

In [ ]:
np.ones_like(df, dtype=np.bool)

In [ ]:
mask = np.triu(np.ones_like(df, dtype=np.bool))

In [ ]:
mask

In [ ]:
new_mask=mask[::-1]
new_mask

In [ ]:
new_new_mask=np.invert(new_mask)

In [ ]:
df_masked=df.mask(new_new_mask)

In [ ]:
df_masked_0=df_masked.fillna(0)

In [ ]:
df_rev=rev_split_fst_stacks

In [ ]:
df_rev

In [ ]:
row_cluster=False
col_cluster=False
xticklabels=x_labels
yticklabels=y_labels
mask=new_new_mask

In [ ]:
cmap = sns.cm.rocket_r

#### **Make the heatmap plot with dendrogram**

In [ ]:
sns.set(font_scale=1.5)

In [ ]:
g = sns.clustermap(df_rev, annot=True, xticklabels=x_labels, yticklabels=y_labels[::-1],figsize=(30, 30), cmap="rocket_r",tree_kws=dict(linewidths=3.5))

In [ ]:
#g.fig.savefig("full_heatmap_with_bold_dengro_diagonal.pdf")

In [ ]:
g1 = sns.clustermap(fst_heat_values, annot=True, xticklabels=x_labels, yticklabels=y_labels,mask=new_mask, figsize=(30, 30),cbar_pos = None)



In [ ]:
#g1.fig.savefig("half_heatmap_row_dendro.png")

In [ ]:
g2 = sns.clustermap(fst_heat_values, annot=True, xticklabels=x_labels, yticklabels=y_labels,row_cluster=False,col_cluster=False,figsize=(30, 30),cbar_pos = None)


In [ ]:
#g2.fig.savefig("half_heatmap_col_dendro.png")